In [2]:
import sys

sys.path.append('..')
from my_util import my_read, my_write

In [3]:
sent_list = my_read('../data/corpus/small_sent.txt')
occur_dict = {line.split(':')[0] : set(map(int, line.split(':')[1].split())) for line in open('../data/corpus/occur.txt', 'r').read().split('\n')}

In [3]:
test_data = my_read('../data/test/co_occur_test.txt')
test_data = [data.split(',') for data in test_data]
test_dict = {data[0] : data[1:] for data in test_data}

In [4]:
test_sent_dict = {central_kw : set() for central_kw in test_dict}
for central_kw, kws in test_dict.items():
    for kw in kws:
        test_sent_dict[central_kw] |= occur_dict[kw]
    test_sent_dict[central_kw] &= occur_dict[central_kw]

for central_kw, sents in test_sent_dict.items():
    content = [sent_list[i] for i in sents]
    my_write('../data/temp/%s_wiki.txt' % central_kw.replace(' ', '_'), content)

In [4]:
test_lines = occur_dict['python'] & (occur_dict['java'] | occur_dict['ruby'])
my_write('python_java_ruby.txt', [sent_list[i] for i in test_lines])

Analyze the sentences with OLLIE, Stanford OpenIE or OpenIE5

In [5]:
openie_data = my_read('../data/temp/pl_wiki_ollie_triple.txt')
# openie_data = my_read('pjr_ollie_triple.txt')
# keywords = set(['data structure', 'binary tree', 'hash table', 'linked list'])
keywords = set(['programming language', 'python', 'java', 'javascript', 'lua', 'scala', 'lisp', 'php', 'ruby', 'smalltalk'])
# keywords = set(['python', 'java', 'ruby'])

qualified_triples = []
for data in openie_data:
    if data:
        arg1, rel, arg2 = data.split(';')
        for kw in keywords:
            if kw in arg1:
                for kw in keywords:
                    if kw in arg2:
                        qualified_triples.append(data)
                        break
                break
my_write('pl_wiki_ollie_triple_f.txt', qualified_triples)

In [4]:
data_structure_idx = occur_dict['data structure']

In [22]:
co_occur_set = {}
for keyword, idx_set in occur_dict.items():
    intersection = idx_set & data_structure_idx
    if intersection:
        co_occur_set[keyword] = list(intersection)

In [39]:
len(co_occur_set)

10125

In [23]:
sorted_co_occur_list = sorted(co_occur_set.items(), key=lambda x: len(x[1]), reverse=True)[:100]
sorted_co_occur_count = [(word, len(idx)) for word, idx in sorted_co_occur_list]

In [30]:
sorted_co_occur_count[:40]

[('data structure', 3804),
 ('data', 3804),
 ('structure', 3804),
 ('time', 716),
 ('query', 630),
 ('algorithm', 537),
 ('space', 427),
 ('tree', 377),
 ('problem', 363),
 ('efficient', 326),
 ('graph', 325),
 ('based', 311),
 ('paper', 300),
 ('present', 299),
 ('set', 281),
 ('using', 260),
 ('model', 243),
 ('dynamic', 232),
 ('memory', 227),
 ('point', 209),
 ('size', 200),
 ('show', 199),
 ('result', 191),
 ('number', 190),
 ('operation', 189),
 ('bound', 185),
 ('use', 181),
 ('given', 178),
 ('application', 164),
 ('linear', 163),
 ('update', 158),
 ('one', 158),
 ('design', 157),
 ('approach', 157),
 ('performance', 153),
 ('search', 148),
 ('technique', 147),
 ('system', 146),
 ('implementation', 143),
 ('propose', 138)]

In [40]:

'b-tree' in co_occur_set

True

In [37]:
# sent_list[co_occur_set['b-tree'][0]]
temp_list = [sent_list[idx] for idx in co_occur_set['b-tree']]
my_write('ds_bt_sent.txt', temp_list)